In [16]:
import twitter
import csv
import json
import re
import requests
import lxml.html as html
from urllib2 import urlopen
import pandas as pd

In [21]:
class TJLoader:

    def __init__(self):
        self._news_pages = "https://tjournal.ru/paper/category/news/page/{}"
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        for i in range(count):

            page = html.parse(urlopen(self._news_pages.format(i+min_index)))
            divs = page.getroot().find_class('b-articles__b__title')

            for div in divs:
                links.append(div.getchildren()[1].get("href"))

        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с tjournal, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__title")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("b-article__infoline__comments")
        comment = int(comments[0].find("b").text.replace(" ", ""))

        # Теги
        tags = root.find_class("b-article__tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infoline__date")
        date = self._parse_date(date[0].text)
        

        return {"title": title, "views": view, "comments": comment, "tags": tag_list, "date": date}


    def get_tj_news_info(self, min_index=10, count=30):
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        for link in links:
            link_info = self.get_link_info(link)
            link = link.split("/")[-1]
            link_info["id"] = link
            link_info_list.append(link_info)

        return link_info_list

In [22]:
loader = TJLoader()
pages = loader.get_tj_news_info(count=1)

In [24]:
pd.DataFrame(pages)

,comments,date,id,tags,title,views
0,20,2015-09-21 10:35,apple-store-viruses,"[Apple, хакеры, приложения для iOS, App Store,...",Apple впервые сообщила о заражении приложений ...,9299
1,9,2015-09-18 19:17,mvd-smart-devices-hack,"[слежка, взломы, слежка за пользователями, умн...",МВД заинтересовалось взломом устройств для «ум...,3946
2,33,2015-09-18 14:08,nuttella-campaign-fail,"[Coca-Cola, Nutella, именные банки]",Пользователи сети саботировали рекламную кампа...,15980
3,97,2015-09-18 11:17,russian-90s,"[Facebook, флешмобы, ретро, 90-е]",Российские журналисты устроили в Фейсбуке фото...,19007
4,21,2015-09-18 10:15,invisible-cloak,"[разработки, университет Беркли, мантия-неведи...",Калифорнийские учёные создали прототип «мантии...,8569
5,4,2015-09-17 17:10,facebook-of-stolen-data,"[хакеры, Китай, киберпреступность, взломы, киб...","СМИ: Китай превратил украденные данные о 21,5 ...",8813
6,21,2015-09-17 14:45,watches-apartment-being-robbed,"[видео, гаджеты, преступность, Canary]",Американец наблюдал за ограблением своего дома...,8181
7,109,2015-09-17 12:27,move-to-ios-android-strikes-back,"[Apple, Android, приложения для Android, Googl...",Фанаты Android «уничтожили» рейтинг приложения...,23471
8,68,2015-09-17 00:05,i-stand-with-ahmed,"[США, школьники, Ахмед Мохамед, самодельные ча...",Пользователи соцсетей устроили флешмоб в подде...,12432
9,49,2015-09-17 00:00,prankers-elton-john-urgant-putin,"[Владимир Путин, пранкеры, Дмитрий Песков, Элт...",Пранкеры рассказали в эфире «Вечернего Урганта...,17821


In [64]:
import twitter
from datetime import datetime, timedelta
from dateutil import tz
import time

class TwitterLoader:


    def __init__(self):
        CONSUMER_KEY = 'BOuuaMDhNhm6yx0rzqK8bMsbI'
        CONSUMER_SECRET = '3DybJwlkXd2vU6R385yLA8yJblYJltLtwojySD9AVs04ShauZ0'

        ACCESS_TOKEN_KEY = '3712177576-of3jzZ8gNmlPDfPjPyR0Ljw1Ao2IXdTqX9dZGDZ'
        ACCESS_TOKEN_SECRET = 'Ky7iKwByHNXX3UMfuMhv6UgVx2IhjLo3KmwpsBQz35wtG'

        self.api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}

        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"

        self._news_uri = "https://tjournal.ru/p/"



    def _parse_date(self, date):

        """

        :param date: str, дата в формате твиттера - "Sat Nov 21 17:00:29 +0000 2015"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        date_array = date.split(' ')
        month = self._month_dict[date_array[1]]
        day = int(date_array[2])
        time = date_array[3]
        year = date_array[5]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
        moscow_date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        return str(moscow_date).split("+")[0]



    def loadTweetWithLink(self, link, date):
        """

        :param link: str, ключевое слово для поиска, вданном случае - ссылка на письмо
        :param date: str, дата, до которой искать твиты
        :return: список словариков с информацией о УНИКАЛЬНЫХ твитах по запросу link
        """
        set_id = set()
        tweet_list = []
        print link, date
        result = self.api.GetSearch(term=link, until=date, count=100000)
        for res in result:

            tw_id = res.GetId()

            # Если мы уже обрабатывали этот твит - идем дальше
            if tw_id in set_id:
                continue

            retweeted_status = res.GetRetweeted_status()
            # Если это не ретвит
            if retweeted_status is None:
                retweeted_count = res.GetRetweetCount()
                favorite_count = res.GetFavoriteCount()
            else:
                retweeted_count = 0
                favorite_count = 0
            set_id.add(tw_id)

            created_at = res.GetCreatedAt()
            created_at = self._parse_date(created_at)

            # Данные о пользователе
            user = res.GetUser()
            followers_count = user.followers_count
            listed_count = user.listed_count
            friends_count = user.friends_count
            favourites_count = user.favourites_count
            statuses_count = user.statuses_count


            link = link.split("/")[-1]

            tw_dict= {
                    "id": link,
                    "tw_id":tw_id,
                    "retweeted_count": retweeted_count,
                    "favorite_count":favorite_count,
                    "created_at":created_at,
                    "user_followers_count":followers_count,
                    "user_listed_count": listed_count,
                    "user_friends_count":friends_count,
                    "user_favourites_count":favourites_count,
                    "user_statuses_count":statuses_count
                    }

            tweet_list.append(tw_dict)

        return tweet_list

    def _get_next_date(self, date, days):
        """

        :param date: str, дата
        :param days: int, количество дней
        :return: возвращает дату через days-дней после date
        """
        date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        date += timedelta(days=days)
        return str(date).split(' ')[0]


    def load_tweets_by_term(self, news_list, days_after_news=2):
        """

        :param news_list: list(str), список словариков с информацией о новости
        :param days_after_news:  int, количество дней после публикации новости, до которой искать
        """
        result_list = []
        for news in news_list:
            tweets = []
            try:
                uri = self._news_uri + news["id"]
                until_date = self._get_next_date(news["date"], days_after_news)
                tweets = self.loadTweetWithLink(uri, until_date)
            except twitter.error.TwitterError as ex:
                print str(ex)
                if str(ex) == self._RATE_LIMIT:
                    sleep_time = self.api.GetSleepTime("statuses/user_timeline")
                    print "Спим {} сек.".format(sleep_time)
                    time.sleep(sleep_time+2)
                    tweets = self.loadTweetWithLink(news)

            result_list.append(tweets)
            
        return result_list





In [65]:
tw = TwitterLoader()

In [ ]:
tw = tw.load_tweets_by_term(pages)

https://tjournal.ru/p/apple-store-viruses 2015-09-23
https://tjournal.ru/p/mvd-smart-devices-hack 2015-09-20
https://tjournal.ru/p/nuttella-campaign-fail 2015-09-20
https://tjournal.ru/p/russian-90s 2015-09-20
https://tjournal.ru/p/invisible-cloak 2015-09-20
https://tjournal.ru/p/facebook-of-stolen-data 2015-09-19
https://tjournal.ru/p/watches-apartment-being-robbed 2015-09-19
https://tjournal.ru/p/move-to-ios-android-strikes-back 2015-09-19
https://tjournal.ru/p/i-stand-with-ahmed 2015-09-19
https://tjournal.ru/p/prankers-elton-john-urgant-putin 2015-09-19
https://tjournal.ru/p/smart-watch-spying 2015-09-18
https://tjournal.ru/p/move-to-ios 2015-09-18
https://tjournal.ru/p/samsung-foldable-phone 2015-09-18
https://tjournal.ru/p/school-clock-bomb 2015-09-18
https://tjournal.ru/p/vladimir-drinkman 2015-09-18
https://tjournal.ru/p/jony-jive 2015-09-17
https://tjournal.ru/p/google-research-balloon-crash 2015-09-17
https://tjournal.ru/p/scissors-robot